In [1]:
import os
import copy

from pyciemss.PetriNetODE.base import MiraPetriNetODESystem, ScaledBetaNoisePetriNetODESystem
from pyciemss.PetriNetODE.events import Event, StartEvent, LoggingEvent, ObservationEvent, StaticParameterInterventionEvent
from pyciemss.Ensemble.base import EnsembleSystem
import pyciemss

from pyciemss.PetriNetODE.interfaces import load_petri_model
from pyciemss.Ensemble.interfaces import setup_model, reset_model, intervene, sample, calibrate, optimize

In [2]:
MIRA_PATH = "test/models/april_ensemble_demo/"

filename1 = "BIOMD0000000955_template_model.json"
filename1 = os.path.join(MIRA_PATH, filename1)
model1 = load_petri_model(filename1, add_uncertainty=True)


# TODO: put this into the interfaces
start_state1 = {k[0]: v.data['initial_value'] for k, v in model1.G.variables.items()}

model1

ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	gamma = Uniform(low: 0.41040000319480896, high: 0.5016000270843506),
	delta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	alpha = Uniform(low: 0.5130000114440918, high: 0.6269999742507935),
	epsilon = Uniform(low: 0.15389999747276306, high: 0.18809999525547028),
	zeta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	XXlambdaXX = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	eta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	rho = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	theta = Uniform(low: 0.33390000462532043, high: 0.4081000089645386),
	kappa = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	mu = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	nu = Uniform(low: 0.024299999698996544, high: 0.02969999983906746),
	xi = Uniform(low: 0.015300000086426735, high: 

In [3]:
filename2 = "BIOMD0000000960_template_model.json"
filename2 = os.path.join(MIRA_PATH, filename2)
model2 = load_petri_model(filename2, add_uncertainty=True)


# TODO: put this into the interfaces
start_state2 = {k[0]: v.data['initial_value'] for k, v in model2.G.variables.items()}

model2

ScaledBetaNoisePetriNetODESystem(
	mira_param_0 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_1 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_2 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_3 = Uniform(low: 0.02098800055682659, high: 0.025652000680565834),
	mira_param_4 = Uniform(low: 0.37501201033592224, high: 0.45834800601005554),
	mira_param_5 = Uniform(low: 0.4526999890804291, high: 0.5533000230789185),
	mira_param_6 = Uniform(low: 0.23669999837875366, high: 0.28929999470710754),
	mira_param_7 = Uniform(low: 0.0027000000700354576, high: 0.0032999999821186066),
	mira_param_8 = Uniform(low: 1.4759999513626099, high: 1.8040000200271606),
	mira_param_9 = Uniform(low: 0.0, high: 0.10000000149011612),
	mira_param_10 = Uniform(low: 0.007199999876320362, high: 0.008799999952316284),
	mira_param_11 = Uniform(low: 0.12690000236034393, high: 0.1551000028848648),
	pseudocount = 1.0
)

In [4]:
from math import isclose

solution_ratio = start_state2['Infectious'] / start_state1['Infected']
solution_mapping1 = lambda x : {"Infected": x["Infected"]}
solution_mapping2 = lambda x: {"Infected": x["Infectious"] / solution_ratio}

# Assert that all of the variables in the solution mappings are the same.
assert(set(solution_mapping1(start_state1).keys()) == set(solution_mapping2(start_state2).keys()))

# Assert that the solution mappings are correct.
assert(isclose(solution_mapping1(start_state1)["Infected"], solution_mapping2(start_state2)["Infected"]))

In [5]:
# JEREMY LOOK HERE.

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)
model3 = load_petri_model(filename3, add_uncertainty=True)

# TODO: put this into the interfaces
start_state3 = {k[0]: v.data['initial_value'] for k, v in model3.G.variables.items()}

model3

ScaledBetaNoisePetriNetODESystem(
	mira_param_0 = Uniform(low: 7.614000097078133e-09, high: 9.306000414710525e-09),
	mira_param_1 = Uniform(low: 1.5228000194156266e-08, high: 1.861200082942105e-08),
	mira_param_2 = Uniform(low: 1.9035000242695332e-09, high: 2.326500103677631e-09),
	mira_param_3 = Uniform(low: 3.8070000485390665e-09, high: 4.653000207355262e-09),
	mira_param_4 = Uniform(low: 0.019285714253783226, high: 0.023571427911520004),
	mira_param_5 = Uniform(low: 0.035999998450279236, high: 0.04399999976158142),
	mira_param_6 = Uniform(low: 0.14399999380111694, high: 0.17599999904632568),
	mira_param_7 = Uniform(low: 0.09000000357627869, high: 0.10999999940395355),
	mira_param_8 = Uniform(low: 0.22499999403953552, high: 0.2750000059604645),
	XXlambdaXX = Uniform(low: 0.27000001072883606, high: 0.33000001311302185),
	theta = Uniform(low: 0.06428571045398712, high: 0.07857142388820648),
	(('Quarantined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C71902')), ('Susceptible_un

In [6]:
# Setup the Ensemble

models = [model1, model2]
weights = [0.5, 0.5]
start_time = 0.0

start_states = [start_state1, start_state2]
solution_mappings = [solution_mapping1, solution_mapping2]

ensemble = setup_model(models, weights, solution_mappings, start_time, start_states)
ensemble

Ensemble of 2 models. 

 	Weights: tensor([0.5000, 0.5000]). 

 	Models: [ScaledBetaNoisePetriNetODESystem(
	beta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	gamma = Uniform(low: 0.41040000319480896, high: 0.5016000270843506),
	delta = Uniform(low: 0.00989999994635582, high: 0.01209999993443489),
	alpha = Uniform(low: 0.5130000114440918, high: 0.6269999742507935),
	epsilon = Uniform(low: 0.15389999747276306, high: 0.18809999525547028),
	zeta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	XXlambdaXX = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	eta = Uniform(low: 0.11249999701976776, high: 0.13750000298023224),
	rho = Uniform(low: 0.03060000017285347, high: 0.03739999979734421),
	theta = Uniform(low: 0.33390000462532043, high: 0.4081000089645386),
	kappa = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	mu = Uniform(low: 0.015300000086426735, high: 0.018699999898672104),
	nu = Uniform(low: 0.024299999698996544, h

In [7]:
# Sample from the Ensemble

timepoints = [1.0, 5.0, 10.0]
num_samples = 2
ensemble_solution = sample(ensemble, timepoints, num_samples)
ensemble_solution


{'model_0/beta': tensor([0.0117, 0.0116]),
 'model_0/gamma': tensor([0.4262, 0.4232]),
 'model_0/delta': tensor([0.0119, 0.0113]),
 'model_0/alpha': tensor([0.5657, 0.5252]),
 'model_0/epsilon': tensor([0.1697, 0.1824]),
 'model_0/zeta': tensor([0.1203, 0.1331]),
 'model_0/XXlambdaXX': tensor([0.0318, 0.0325]),
 'model_0/eta': tensor([0.1182, 0.1196]),
 'model_0/rho': tensor([0.0351, 0.0326]),
 'model_0/theta': tensor([0.3922, 0.3962]),
 'model_0/kappa': tensor([0.0163, 0.0165]),
 'model_0/mu': tensor([0.0175, 0.0160]),
 'model_0/nu': tensor([0.0247, 0.0286]),
 'model_0/xi': tensor([0.0165, 0.0157]),
 'model_0/tau': tensor([0.0102, 0.0095]),
 'model_0/sigma': tensor([0.0173, 0.0184]),
 'model_1/mira_param_0': tensor([0.0260, 0.0607]),
 'model_1/mira_param_1': tensor([0.0729, 0.0428]),
 'model_1/mira_param_2': tensor([0.0791, 0.0884]),
 'model_1/mira_param_3': tensor([0.0232, 0.0225]),
 'model_1/mira_param_4': tensor([0.3826, 0.4567]),
 'model_1/mira_param_5': tensor([0.5137, 0.4648]),


In [8]:
data = [(1.1, {"Infected": 0.003}), (1.2, {"Infected": 0.005})]

# TODO: increase the number of iterations
inferred_parameters = calibrate(ensemble, data, num_iterations=100, verbose=True)

iteration 0: loss = 87.72814571857452


In [ ]:
posterior_predictive = sample(ensemble, timepoints, num_samples, inferred_parameters)
posterior_predictive

{'model_0/beta': tensor([0.0104, 0.0115]),
 'model_0/gamma': tensor([0.4795, 0.4929]),
 'model_0/delta': tensor([0.0109, 0.0116]),
 'model_0/alpha': tensor([0.5467, 0.6214]),
 'model_0/epsilon': tensor([0.1701, 0.1735]),
 'model_0/zeta': tensor([0.1325, 0.1242]),
 'model_0/XXlambdaXX': tensor([0.0340, 0.0315]),
 'model_0/eta': tensor([0.1267, 0.1131]),
 'model_0/rho': tensor([0.0324, 0.0321]),
 'model_0/theta': tensor([0.3431, 0.3572]),
 'model_0/kappa': tensor([0.0167, 0.0171]),
 'model_0/mu': tensor([0.0174, 0.0166]),
 'model_0/nu': tensor([0.0277, 0.0258]),
 'model_0/xi': tensor([0.0181, 0.0181]),
 'model_0/tau': tensor([0.0107, 0.0094]),
 'model_0/sigma': tensor([0.0160, 0.0174]),
 'model_1/mira_param_0': tensor([0.0270, 0.0085]),
 'model_1/mira_param_1': tensor([0.0350, 0.0062]),
 'model_1/mira_param_2': tensor([0.0351, 0.0659]),
 'model_1/mira_param_3': tensor([0.0243, 0.0251]),
 'model_1/mira_param_4': tensor([0.3824, 0.3990]),
 'model_1/mira_param_5': tensor([0.5456, 0.5423]),


## MIRA Explorations below

In [ ]:
import json
import mira

filename1 = "BIOMD0000000955_template_model.json"
filename1 = os.path.join(MIRA_PATH, filename1)

with open(filename1, "r") as f:
    model_json1 = json.load(f)

template1 = mira.metamodel.TemplateModel.from_json(model_json1)
model1 = mira.modeling.Model(template1)

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

template3 = mira.metamodel.TemplateModel.from_json(model_json3)
model3 = mira.modeling.Model(template3)
# G = pyciemss.PetriNetODE.base.PetriNet.from_mira(filename3)

# model3 = MiraPetriNetODESystem.from_mira(filename3)
# model3 = ScaledBetaNoisePetriNetODESystem.from_mira(filename3)
# model3

In [ ]:
from mira.metamodel.ops import simplify_rate_laws, aggregate_parameters

In [8]:
template1.parameters

{'alpha': Parameter(name='alpha', identifiers={}, context={}, value=0.57, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.45599999999999996, 'maximum': 0.6839999999999999})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'gamma': Parameter(name='gamma', identifiers={}, context={}, value=0.456, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472})),
 'delta': Parameter(name='delta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'epsilon': Parameter(name='epsilon', identifiers={}, context={}, value=0.171, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.1368, 'maximum': 0.20520000000000002})),
 'theta': Parameter(name='theta', iden

In [16]:
new_template = simplify_rate_laws(aggregate_parameters(template1))
# new_template = simplify_rate_laws(template1)
new_template.parameters



{'alpha': Parameter(name='alpha', identifiers={}, context={}, value=0.57, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.45599999999999996, 'maximum': 0.6839999999999999})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'gamma': Parameter(name='gamma', identifiers={}, context={}, value=0.456, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.3648, 'maximum': 0.5472})),
 'delta': Parameter(name='delta', identifiers={}, context={}, value=0.011, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.008799999999999999, 'maximum': 0.0132})),
 'epsilon': Parameter(name='epsilon', identifiers={}, context={}, value=0.171, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.1368, 'maximum': 0.20520000000000002})),
 'theta': Parameter(name='theta', iden

In [12]:

template3.parameters

{'f': Parameter(name='f', identifiers={}, context={}, value=0.2, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.16, 'maximum': 0.24000000000000002})),
 'n': Parameter(name='n', identifiers={}, context={}, value=0.5, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.4, 'maximum': 0.6})),
 'q': Parameter(name='q', identifiers={}, context={}, value=0.83, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.6639999999999999, 'maximum': 0.996})),
 'sigma': Parameter(name='sigma', identifiers={}, context={}, value=0.2, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 0.16, 'maximum': 0.24000000000000002})),
 'beta': Parameter(name='beta', identifiers={}, context={}, value=2.115e-08, distribution=Distribution(type='StandardUniform1', parameters={'minimum': 1.6920000000000002e-08, 'maximum': 2.538e-08})),
 'mu': Parameter(name='mu', identifiers={}, context={}, value=0.2, distribution=Distribu

In [22]:
from mira.metamodel.ops import aggregate_parameters

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

template3 = mira.metamodel.TemplateModel.from_json(model_json3)
model3 = mira.modeling.Model(template3)

new_template = aggregate_parameters(template3)
new_model = mira.modeling.Model(new_template)
new_model.parameters

{'mira_param_0': <mira.modeling.ModelParameter at 0x7fee91669960>,
 'mira_param_1': <mira.modeling.ModelParameter at 0x7fee91669de0>,
 'mira_param_2': <mira.modeling.ModelParameter at 0x7fee91669d50>,
 'mira_param_3': <mira.modeling.ModelParameter at 0x7fee916694b0>,
 'mira_param_4': <mira.modeling.ModelParameter at 0x7fee91669450>,
 'mira_param_5': <mira.modeling.ModelParameter at 0x7fee91669360>,
 'mira_param_6': <mira.modeling.ModelParameter at 0x7fee91669330>,
 'mira_param_7': <mira.modeling.ModelParameter at 0x7fee91669240>,
 'mira_param_8': <mira.modeling.ModelParameter at 0x7fee91669210>,
 'XXlambdaXX': <mira.modeling.ModelParameter at 0x7fee91669f30>,
 'theta': <mira.modeling.ModelParameter at 0x7fee91669150>,
 (('Quarantined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C71902')),
  ('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  'NaturalConversion',
  'rate'): <mira.modeling.ModelParameter at 0x7fee916690f0>,
 'mira_para

In [26]:
model1.parameters

{'beta': <mira.modeling.ModelParameter at 0x7fcb7c5388e0>,
 'gamma': <mira.modeling.ModelParameter at 0x7fcb7c53bc10>,
 'delta': <mira.modeling.ModelParameter at 0x7fcb7c53a050>,
 'alpha': <mira.modeling.ModelParameter at 0x7fcb7c5e9570>,
 'epsilon': <mira.modeling.ModelParameter at 0x7fcb7c5eb430>,
 'zeta': <mira.modeling.ModelParameter at 0x7fcb7c5eac80>,
 'XXlambdaXX': <mira.modeling.ModelParameter at 0x7fcb7c5eabc0>,
 'eta': <mira.modeling.ModelParameter at 0x7fcb7c5eab00>,
 'rho': <mira.modeling.ModelParameter at 0x7fcb7c5eaa70>,
 'theta': <mira.modeling.ModelParameter at 0x7fcb7c5ea9e0>,
 'kappa': <mira.modeling.ModelParameter at 0x7fcb7c5ea950>,
 'mu': <mira.modeling.ModelParameter at 0x7fcb7c5ea890>,
 'nu': <mira.modeling.ModelParameter at 0x7fcb7c5ea7d0>,
 'xi': <mira.modeling.ModelParameter at 0x7fcb7c5ea740>,
 'tau': <mira.modeling.ModelParameter at 0x7fcb7c5ea680>,
 'sigma': <mira.modeling.ModelParameter at 0x7fcb7c5ea5c0>}

In [44]:
model3.transitions

{(('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Exposed', ('identity', 'apollosv:00000154')),
  ('Infected_reported',
   ('identity', 'ido:0000511'),
   ('diagnosis', 'ncit:C15220')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8d00>,
 (('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Exposed', ('identity', 'apollosv:00000154')),
  ('Infected_unreported',
   ('identity', 'ido:0000511'),
   ('diagnosed', 'ncit:C113725')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8c10>,
 (('Susceptible_unconfined',
   ('identity', 'ido:0000514'),
   ('quarantined', 'ncit:C68851')),
  ('Quarantined', ('identity', 'ido:0000514'), ('quarantined', 'ncit:C71902')),
  ('Infected_reported',
   ('identity', 'ido:0000511'),
   ('diagnosis', 'ncit:C15220')),
  'ControlledConversion'): <mira.modeling.Transition at 0x7fcb7c5e8b20>,
 (('Susceptible_unconfined',
   ('ide

In [42]:
from mira.modeling.ode import OdeModel

ode1 = OdeModel(model1)

ode1.set_parameters({'beta': 0.5})

ode1.kinetics
ode1.get_rhs()

<function mira.modeling.ode.OdeModel.get_rhs.<locals>.rhs(t, y)>

In [35]:


list(model3.transitions.values())[0].template

ControlledConversion(rate_law=Infected_reported*Susceptible_unconfined*beta*n*(1.0 - sigma), type='ControlledConversion', controller=Concept(name='Infected_reported', identifiers={'ido': '0000511'}, context={'diagnosis': 'ncit:C15220'}), subject=Concept(name='Susceptible_unconfined', identifiers={'ido': '0000514'}, context={'quarantined': 'ncit:C68851'}), outcome=Concept(name='Exposed', identifiers={'apollosv': '00000154'}, context={}), provenance=[])

In [13]:
import json
import mira

filename3 = "BIOMD0000000983_template_model.json"
filename3 = os.path.join(MIRA_PATH, filename3)

with open(filename3, "r") as f:
    model_json3 = json.load(f)

example_template = mira.metamodel.TemplateModel.from_json(model_json3)

example_model = mira.modeling.Model(example_template)

In [14]:
for p, v in example_template.parameters.items():
    print(p, v.value)
    

f 0.2
n 0.5
q 0.83
sigma 0.2
beta 2.115e-08
mu 0.2
eta_r 0.1
eta_u 0.25
theta 0.07142857
_1 1.0
XXlambdaXX 0.3
h 0.05


In [ ]:
for p, v in example_template.parameters.items():
    print(p, v.value)